In [1]:
import os
import cv2
import numpy as np

In [2]:
# Load the face cascade classifier
face_cascade = cv2.CascadeClassifier("./lbpcascade_frontalface.xml")

input_dir = "./raw_dataset/"
output_dir = "./preprocessed_dataset/"

# Loop over the input images and process each one
for name in os.listdir(input_dir):
    if os.path.isdir(os.path.join(input_dir, name)):
        # Create the output directory if it doesn't exist
        output_folder = os.path.join(output_dir, name)
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
    for filename in os.listdir(os.path.join(input_dir, name)):
        if filename.endswith('.jpg'):
            # Load the input image
            img = cv2.imread(os.path.join(input_dir, name, filename))
            
            # print(img.dtype)
            # print(img.shape)
            
            # Check if the image is empty
            if img is None:
                continue
    
            # Convert the image to grayscale
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
            # Detect the faces in the image using the face cascade classifier
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
            
            # Skip the image if no faces are detected
            if len(faces) == 0:
                continue
            
            # Iterate over the detected faces and align them
            for (x, y, w, h) in faces:
                # Crop the face region
                face_img = gray[y:y+h, x:x+w]
        
                # Resize the face image to a fixed size (e.g., 180x180)
                face_img = cv2.resize(face_img, (120, 120))
        
                # Normalize the pixel values of the face image to be between 0 and 1
                face_img = face_img.astype(float) / 255.0
        
                # Define the gamma value
                gamma = 1.5

                # Apply gamma correction
                corrected = cv2.pow(face_img/255.0, gamma)

                # Normalize the output image
                face_img = cv2.normalize(corrected, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

                # Save the aligned face image to the output directory
                output_filename = os.path.join(output_folder, filename)
                cv2.imwrite(output_filename, face_img)

In [3]:
from skimage.feature import hog, local_binary_pattern
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Set the input directory and load the pre-processed face images
input_dir = "./preprocessed_dataset/"
batch_size = 1000

# Initialize the feature vectors and labels
face_features = []
face_labels = []

for batch_num, name in enumerate(os.listdir(input_dir)):
    if not os.path.isdir(os.path.join(input_dir, name)):
        continue
    
    # Load the batch of images
    batch_images = []
    for filename in os.listdir(os.path.join(input_dir, name)):
        if filename.endswith(".jpg"):
            # Load the pre-processed face image and extract the label from the filename
            img = cv2.imread(os.path.join(input_dir, name, filename), cv2.IMREAD_GRAYSCALE)
            if img is None: 
                print(f"Error loading image: {os.path.join(input_dir, name, filename)}")
                continue
            batch_images.append(img)
            
    # Define the HOG parameters
    win_size = (120, 120)
    block_size = (8, 8)
    block_stride = (4, 4)
    cell_size = (4, 4)
    num_bins = 9

    # Define the LBP parameters
    radius = 3
    num_points = 8 * radius
    
    # Extract HOG and LBP features for the batch of images
    hog_features = []
    lbp_features = []
    for img in batch_images:
        # Extract HOG features
        hog_feat = hog(img, orientations=num_bins, pixels_per_cell=cell_size, cells_per_block=block_size, block_norm='L2-Hys', feature_vector=True)
        hog_features.append(hog_feat)

        # Extract LBP features
        lbp_feat = local_binary_pattern(img, num_points, radius, method='uniform')
        lbp_features.append(lbp_feat.flatten())
        
    # Combine the HOG and LBP features for the batch
    hog_features = np.array(hog_features)
    lbp_features = np.array(lbp_features)
    
    # print("HOG features shape:", hog_features.shape)
    # print("LBP features shape:", lbp_features.shape)
    
    if len(hog_features) == 0 or len(lbp_features) == 0:
    # Skip this batch if either of the arrays is empty
        continue

    # Concatenate the arrays along axis 1
    features_batch = np.concatenate([hog_features, lbp_features], axis=1)
    face_features.append(features_batch)
    
    # Extract the label for the batch
    labels_batch = []
    for filename in os.listdir(os.path.join(input_dir, name)):
        if filename.endswith(".jpg"):
            split_filename = filename.split("_")
            if len(split_filename) < 2:
                print(f"Error processing filename: {os.path.join(input_dir, name, filename)}")
                continue  # skip filenames that don't contain at least two underscores
            firstname, lastname = split_filename[:2]
            if len(split_filename) == 2:
                label = f"{firstname}"
            else:
                label = f"{firstname}_{lastname}"
            labels_batch.append(label)
            
    # if len(labels_batch) != len(batch_images):
    #     continue # skip batches with inconsistent numbers of labels
    
    face_labels.append(labels_batch)
    
     # If we've processed a full batch, train the SVM model and evaluate it
    if (batch_num + 1) % batch_size == 0 or batch_num == len(os.listdir(input_dir)) - 1:
        # Flatten the features and labels arrays
        features_flat = np.concatenate(face_features, axis=0)
        labels_flat = np.concatenate(face_labels, axis=0)

        # Split the data into training, validation, and testing sets
        X_train, X_test, y_train, y_test = train_test_split(features_flat, labels_flat, test_size=0.2, random_state=42)
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)
        
        # Print the shapes of the training, validation, and testing sets
        print("Training set shape:", X_train.shape)
        print("Validation set shape:", X_val.shape)
        print("Testing set shape:", X_test.shape)

        # Train an SVM model on the training set
        svm_model = SVC(kernel='linear')
        svm_model.fit(X_train, y_train)

        # Evaluate the SVM classifier on the training, validation, and testing sets
        y_pred_train = svm_model.predict(X_train)
        train_acc = accuracy_score(y_train, y_pred_train)
        y_pred_val = svm_model.predict(X_val)
        val_acc = accuracy_score(y_val, y_pred_val)
        print(f"Batch {batch_num + 1}: Training accuracy = {train_acc:.2f}, Validation accuracy = {val_acc:.2f}")
        
        # Evaluate the model on the testing set
        y_pred_test = svm_model.predict(X_test)
        test_acc = accuracy_score(y_test, y_pred_test)
        test_prec = precision_score(y_test, y_pred_test, average='weighted')
        test_rec = recall_score(y_test, y_pred_test, average='weighted')
        test_f1 = f1_score(y_test, y_pred_test, average='weighted')
        
        print("Test accuracy:", test_acc)
        print("Test precision:", test_prec)
        print("Test recall:", test_rec)
        print("Test F1 score:", test_f1)
        
        # Clear the features and labels arrays for the next batch
        features = []
        labels = []

Training set shape: (1156, 319104)
Validation set shape: (386, 319104)
Testing set shape: (386, 319104)
Batch 1000: Training accuracy = 1.00, Validation accuracy = 0.09


d:\finalyearproject\hajirilab\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\finalyearproject\hajirilab\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Test accuracy: 0.06217616580310881
Test precision: 0.022151455387790495
Test recall: 0.06217616580310881
Test F1 score: 0.019939990924446885


MemoryError: Unable to allocate 2.32 MiB for an array with shape (23, 23, 8, 8, 9) and data type float64

In [ ]:
import matplotlib.pyplot as plt

# Visualize results
fig, axs = plt.subplots(3, 3, figsize=(8, 8))
axs = axs.ravel()

for i, (img, pred_label, true_label) in enumerate(zip(batch_images, y_pred_test, y_test)):
    axs[i].imshow(X_test[i], cmap='gray')
    axs[i].set_title(f"Predicted label: {pred_label}\nTrue label: {true_label}")
    axs[i].axis('off')
    
    if i == 8:  # break out of loop when we've plotted 9 images
        break

plt.tight_layout()

In [ ]:
# Save the trained model
import joblib

joblib.dump(clf, 'face_recognition_model.pkl')